In [37]:
#For importing modules
import sys


# final pre-processing
from imblearn.over_sampling import SMOTE
from sklearn.utils import shuffle
from tensorflow.keras.utils import Sequence
import numpy as np


# model
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout, Input, BatchNormalization

# evaluation
from sklearn.metrics import precision_score, recall_score, confusion_matrix,classification_report


In [4]:
sys.path.append('/path/to/2014_07_13_test')
import generate_input

## Load data

In [6]:
# filter small cases
filter_case = ['case_00018', 'case_00031', 'case_00042', 'case_00056', 'case_00088', 'case_00090', 'case_00092', 
               'case_00106', 'case_00120', 'case_00133', 'case_00183', 'case_00200', 'case_00211', 'case_00228',
               'case_00234', 'case_00237', 'case_00249', 'case_00277', 'case_00288', 'case_00291']

In [8]:
# load validation set
val_case = ['case_00233', 'case_00089', 'case_00050', 'case_00112', 'case_00258', 'case_00246', 'case_00157',
            'case_00149','case_00184']
test_case = ['case_00221', 'case_00259', 'case_00087', 'case_00254', 'case_00098', 'case_00023', 'case_00041',
             'case_00080', 'case_00101', 'case_00164', 'case_00002', 'case_00110', 'case_00030', 'case_00068',
             'case_00026', 'case_00063', 'case_00006', 'case_00048', 'case_00250', 'case_00238', 'case_00015',
             'case_00111', 'case_00278', 'case_00133', 'case_00284', 'case_00282', 'case_00269', 'case_00039',
             'case_00033', 'case_00108', 'case_00175', 'case_00161', 'case_00256', 'case_00119', 'case_00286',
             'case_00077', 'case_00162', 'case_00270', 'case_00271', 'case_00285', 'case_00174', 'case_00147',
             'case_00215', 'case_00150', 'case_00052', 'case_00231', 'case_00198', 'case_00117', 'case_00138',
             'case_00211', 'case_00190', 'case_00248', 'case_00235', 'case_00049', 'case_00074', 'case_00107',
             'case_00218', 'case_00001', 'case_00193', 'case_00067', 'case_00289', 'case_00072', 'case_00044',
             'case_00294', 'case_00298', 'case_00263', 'case_00038', 'case_00299', 'case_00249', 'case_00225',
             'case_00217', 'case_00178', 'case_00082', 'case_00035', 'case_00034', 'case_00047', 'case_00276',
             'case_00151', 'case_00226', 'case_00086', 'case_00176']

In [9]:
all_cases = [ 'case_' + str(number).zfill(5) for number in range(300)] # create list of all cases
train_case = [x for x in all_cases if x not in val_case and x not in test_case] # take only cases not in test and val

In [10]:
def generate_part_label_dict(labels_dict, list_of_cases):
    return {k: labels_dict[k] for k in list_of_cases}

def load_data(rootdir, img_dir, json_path, list_cases):
    val_path = generate_input.processed_image_paths(rootdir, img_dir)
    labels_dict = generate_input.malignant_labels_to_dict(json_path)
    labels_dict_short = generate_part_label_dict(labels_dict, list_cases)
    image_dict_short = generate_input.load_nifti_img_and_mask_as_numpy(val_path, list_cases)
    x_set, y_set = generate_input.fill_set(list_cases, labels_dict_short, image_dict_short, labels_dict)
    x_set = generate_input.adding_channel(x_set)
    return x_set, y_set

In [11]:
x_val, y_val = load_data('./preprocessed-data/', 'images', '../kits21/kits21/data/kits.json', val_case)
x_test, y_test = load_data('./preprocessed-data/', 'images', '../kits21/kits21/data/kits.json', test_case)

In [12]:
x_train, y_train = load_data('./preprocessed-data/', 'images', '../kits21/kits21/data/kits.json', train_case)

In [ ]:
# x_train, y_train, x_val, y_val, x_test, y_test, id_val, id_test = generate_input.generate_data_input(
#     '../kits21/kits21/data/kits.json', n_samples = 300, neg_pct = 100, rootdir = './preprocessed-data/', 
#     img_dir = 'images', mask_dir = 'masks', val_split = 9, test_split = 81)

## Upsampling 

In [16]:
x_train_reshape = x_train.reshape(x_train.shape[0], x_train.shape[1] * x_train.shape[2] * x_train.shape[3])
sm = SMOTE(random_state=64)
x_smote, y_smote = sm.fit_resample(x_train_reshape, y_train)
x_smote = x_smote.reshape(x_smote.shape[0], x_train.shape[1], x_train.shape[2], x_train.shape[3])

In [20]:
x_smote, y_smote = shuffle(x_smote, y_smote)

## Developing baseline model

In [21]:
# a code is taken from https://stackoverflow.com/questions/62916904/failed-copying-input-tensor-from-cpu-to-gpu-in-order-to-run-gatherve-dst-tensor

batch_size = 32
class DataGenerator(Sequence):
    """ Send data to the model using TF in specified sizes of batch """
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

train_gen = DataGenerator(x_smote, y_smote, batch_size)

In [27]:
# draft model
#inputs = Input(shape=(224, 224, 3))
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=2)

resnet = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(128,128,3))

for layer in resnet.layers[:170]:
    layer.trainable = False

metrics_list = [tf.keras.metrics.AUC(name = 'auc'),
                tf.keras.metrics.BinaryAccuracy(name = 'accuracy')]

#calculate class weights
class_weights = {0 : 25, 1 : 1} 

optimizer_fn = tf.keras.optimizers.experimental.RMSprop(learning_rate=0.001, jit_compile = False)
#optimizer_fn = tf.keras.optimizers.Adam(learning_rate=0.00002)

x = Flatten()(resnet.output)
x = Dense(1024, activation = 'relu', kernel_regularizer='l2')(x)
x = Dropout(0.3)(x)
x = Dense(512, activation = 'relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation = 'relu')(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
x = Dense(128, activation = 'relu', kernel_regularizer='l2')(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
x = Dense(64, activation = 'relu')(x)
x = Dense(1, activation = 'sigmoid')(x)

model = tf.keras.Model(resnet.input, x)

model.compile(optimizer = optimizer_fn, loss='binary_crossentropy', metrics= metrics_list)
model.summary()
model.fit(train_gen, validation_data = (x_val, y_val), epochs=5, batch_size = batch_size, class_weight = class_weights)


In [75]:
results = model.evaluate(x_val, y_val)
results

58/58 [==============================] - 3s 46ms/step - loss: 1.4998 - auc: 0.5277 - accuracy: 0.9321


[1.4998470544815063, 0.5276820063591003, 0.9321120977401733]

In [76]:
y_pred = model.predict(x_val)

2023-04-30 19:52:13.470334: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


58/58 [==============================] - 5s 47ms/step


In [77]:
precision = precision_score(y_val, y_pred.round()) 
recall = recall_score(y_val, y_pred.round())
print(precision)
print(recall)

0.9417528579205225
0.989136649514008


In [78]:
confusion_matrix(y_val, y_pred.round())

array([[   0,  107],
       [  19, 1730]])

In [79]:
print(classification_report(y_val, y_pred.round()))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       107
         1.0       0.94      0.99      0.96      1749

    accuracy                           0.93      1856
   macro avg       0.47      0.49      0.48      1856
weighted avg       0.89      0.93      0.91      1856



## Evaluation of test set

In [25]:
y_pred_test = model.predict(x_test)

537/537 [==============================] - 22s 40ms/step


In [26]:
confusion_matrix(y_test, y_pred_test.round())

array([[  271,   528],
       [ 4680, 11681]])

In [27]:
print(classification_report(y_test, y_pred_test.round()))

              precision    recall  f1-score   support

         0.0       0.05      0.34      0.09       799
         1.0       0.96      0.71      0.82     16361

    accuracy                           0.70     17160
   macro avg       0.51      0.53      0.46     17160
weighted avg       0.91      0.70      0.78     17160



## Save model

In [29]:
model_json = model.to_json()
with open("model1_binary2.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model1_binary2.h5")

## Load model

In [29]:
# load json and create model
json_file = open('baseline_model/model1_binary2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = tf.keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("baseline_model/model1_binary2.h5")

In [30]:
# evaluate loaded model on test data
optimizer_fn = tf.keras.optimizers.experimental.RMSprop(learning_rate=0.001, jit_compile = False)
metrics_list = [tf.keras.metrics.AUC(name = 'auc'),
                tf.keras.metrics.BinaryAccuracy(name = 'accuracy')]
loaded_model.compile(optimizer = optimizer_fn, loss='binary_crossentropy', metrics= metrics_list)
score1 = loaded_model.evaluate(x_val, y_val)
score1

2023-05-01 11:45:52.826066: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


58/58 [==============================] - 4s 46ms/step - loss: 1.0652 - auc: 0.6680 - accuracy: 0.6972


[1.0651888847351074, 0.6679624319076538, 0.6971982717514038]

## Majority voting

In [31]:
# load validation set
val_case = ['case_00233', 'case_00089', 'case_00050', 'case_00112', 'case_00258', 'case_00246', 'case_00157', 'case_00149','case_00184']
test_case = ['case_00221', 'case_00259', 'case_00087', 'case_00254', 'case_00098', 'case_00023', 'case_00041',
             'case_00080', 'case_00101', 'case_00164', 'case_00002', 'case_00110', 'case_00030', 'case_00068',
             'case_00026', 'case_00063', 'case_00006', 'case_00048', 'case_00250', 'case_00238', 'case_00015',
             'case_00111', 'case_00278', 'case_00133', 'case_00284', 'case_00282', 'case_00269', 'case_00039',
             'case_00033', 'case_00108', 'case_00175', 'case_00161', 'case_00256', 'case_00119', 'case_00286',
             'case_00077', 'case_00162', 'case_00270', 'case_00271', 'case_00285', 'case_00174', 'case_00147',
             'case_00215', 'case_00150', 'case_00052', 'case_00231', 'case_00198', 'case_00117', 'case_00138',
             'case_00211', 'case_00190', 'case_00248', 'case_00235', 'case_00049', 'case_00074', 'case_00107',
             'case_00218', 'case_00001', 'case_00193', 'case_00067', 'case_00289', 'case_00072', 'case_00044',
             'case_00294', 'case_00298', 'case_00263', 'case_00038', 'case_00299', 'case_00249', 'case_00225',
             'case_00217', 'case_00178', 'case_00082', 'case_00035', 'case_00034', 'case_00047', 'case_00276',
             'case_00151', 'case_00226', 'case_00086', 'case_00176']

In [49]:
def adding_channel_dict(image_dict):
    """Convert from one channel to three channel images that are stored in dictionary """
    for key in image_dict:
        image_dict[key] = generate_input.adding_channel(image_dict[key])
    return image_dict

def load_data_to_evaluate_by_majority_voting(rootdir, img_dir, json_path, list_cases):
    """ Load images and labels in dictionary format """
    val_path = generate_input.processed_image_paths(rootdir, img_dir)
    labels_dict = generate_input.malignant_labels_to_dict(json_path)
    labels_dict_short = generate_part_label_dict(labels_dict, list_cases)
    labels_dict_short = {key: float(value) for key, value in labels_dict_short.items()}
    
    image_dict_short = generate_input.load_nifti_img_and_mask_as_numpy(val_path, list_cases)
    image_dict_short = adding_channel_dict(image_dict_short)
    return image_dict_short, labels_dict_short
    
def create_prediction_dict(image_dict, model):
    """ Create dictionary with prediction on patient-level using majority voting """
    pred_dict = {}
    for key in image_dict:
        y_pred = model.predict(image_dict[key])
        pred_dict[key] = np.round(np.sum(y_pred)/len(y_pred))
    return pred_dict

def create_true_and_prediction_lists(true_dict, pred_dict):
    """ Convert dictionaries into lists for evaluation """
    common_keys = true_dict.keys() & pred_dict.keys()
    true_lst = []
    pred_lst = [] 
    for key in common_keys:
        true_lst.append(true_dict[key])
        pred_lst.append(pred_dict[key])   
    return true_lst, pred_lst

### Validation set

In [33]:
val_dict, labels_val = load_data_to_evaluate_by_majority_voting('./preprocessed-data/', 'images', '../kits21/kits21/data/kits.json', val_case)

In [50]:
val_pred_dict = create_prediction_dict(val_dict, loaded_model)

5/5 [==============================] - 0s 39ms/step


In [39]:
val_true_lst, val_pred_lst = create_true_and_prediction_lists(labels_val, val_pred_dict)

In [40]:
print(confusion_matrix(val_true_lst, val_pred_lst))

[[0 1]
 [2 6]]


In [41]:
print(classification_report(val_true_lst, val_pred_lst))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         1
         1.0       0.86      0.75      0.80         8

    accuracy                           0.67         9
   macro avg       0.43      0.38      0.40         9
weighted avg       0.76      0.67      0.71         9



### Test set

In [42]:
test_dict, test_label_dict = load_data_to_evaluate_by_majority_voting('./preprocessed-data/', 'images', '../kits21/kits21/data/kits.json', test_case)

In [43]:
test_pred_dict = create_prediction_dict(test_dict, loaded_model)

3/3 [==============================] - 0s 90ms/step


In [44]:
test_true_lst, test_pred_lst = create_true_and_prediction_lists(test_label_dict, test_pred_dict)

In [45]:
print(confusion_matrix(test_true_lst, test_pred_lst))

[[ 2  5]
 [11 63]]


In [46]:
print(classification_report(test_true_lst, test_pred_lst))

              precision    recall  f1-score   support

         0.0       0.15      0.29      0.20         7
         1.0       0.93      0.85      0.89        74

    accuracy                           0.80        81
   macro avg       0.54      0.57      0.54        81
weighted avg       0.86      0.80      0.83        81

